In [6]:
import os
import requests
import pandas as pd
import json

In [7]:
with open('idtoken.json', 'r', encoding='utf-8') as f:
    idtoken = json.load(f)

idToken = idtoken['idToken']
# ulr = 'https://api.jquants.com/v1/prices/daily_quotes'

In [8]:

# パラメータとして銘柄コード、取得開始日、終了日などを指定
stock_code = "7203"  # トヨタ自動車の例
start_date = "2023-01-01"
end_date = "2023-12-31"
url = f"https://api.jquants.com/v1/prices/daily_quotes?code={stock_code}&from={start_date}&to={end_date}"

# ヘッダーにAPIキーを付与
headers = {'Authorization': 'Bearer {}'.format(idToken)}

# リクエスト送信
response = requests.get(url, headers=headers)

if response.status_code == 200:
    # JSONレスポンスを取得
    data = response.json()
    # 返されるJSONデータのキーに合わせて変換
    # 例: {"daily_quotes": [{"date": "...", "open": ..., "high": ..., "low": ..., "close": ..., "volume": ...}, ...]}
    if "daily_quotes" in data:
        df = pd.DataFrame(data["daily_quotes"])
        print("データ取得成功")
        print(df.head())
    else:
        print("想定するデータキー 'daily_quotes' が見つかりません:", data)
else:
    print("APIアクセスに失敗:", response.status_code, response.text)

データ取得成功
         Date   Code    Open    High     Low   Close UpperLimit LowerLimit  \
0  2023-01-04  72030  1798.0  1804.0  1787.5  1799.0          0          0   
1  2023-01-05  72030  1812.0  1819.5  1793.5  1807.5          0          0   
2  2023-01-06  72030  1809.5  1829.0  1806.0  1825.0          0          0   
3  2023-01-10  72030  1837.5  1850.0  1821.5  1827.0          0          0   
4  2023-01-11  72030  1824.0  1840.0  1822.0  1837.5          0          0   

       Volume  TurnoverValue  AdjustmentFactor  AdjustmentOpen  \
0  25995600.0   4.673681e+10               1.0          1798.0   
1  24700200.0   4.458720e+10               1.0          1812.0   
2  22568600.0   4.106692e+10               1.0          1809.5   
3  22352300.0   4.093154e+10               1.0          1837.5   
4  19798400.0   3.630799e+10               1.0          1824.0   

   AdjustmentHigh  AdjustmentLow  AdjustmentClose  AdjustmentVolume  
0          1804.0         1787.5           1799.0       

In [10]:

# 例としてdfは以下のカラムを持つとする: date, open, high, low, close, volume
# df = ... (前ステップで取得、整形済み)

# 日付でソートしておく
df = df.sort_values("Date").reset_index(drop=True)

# 欠損値の確認
print("欠損値数:\n", df.isnull().sum())

# 欠損値補間例: 
# ここではOHLCVに欠損があった場合、前日値で補間(前方フィル)する例を示す
df[['Open','High','Low','Close','Volume']] = df[['Open','High','Low','Close','Volume']].ffill()

# 再度欠損値確認
print("欠損値補間後:\n", df.isnull().sum())

# 異常値チェック: 
# 1) 例えばvolumeが負になったり、極端に大きすぎる値がないかを確認
# 基本的には出来高が負になることはありえないので、それをチェック
if (df['Volume'] < 0).any():
    # volumeが負の行を出力して、対処
    print("負の出来高が検出されました。対象行:\n", df[df['Volume'] < 0])
    # 負の出来高は欠損として扱い、補間するか、その行を削除
    df.loc[df['Volume'] < 0, 'Volume'] = np.nan
    df['Volume'] = df['Volume'].ffill()

# 2) 価格の異常値チェック: 
# 極端なスパイク（過去平均値から10σ以上離れるような値）がないか確認
# まずはclose価格について標準偏差ベースで簡易チェック

mean_close = df['Close'].mean()
std_close = df['Close'].std()
threshold = mean_close + 10 * std_close  # 極端な閾値の例

outliers = df[df['Close'] > threshold]
if not outliers.empty:
    print("異常な高値が検出されました:", outliers)
    # 対策例: 異常値を直前の値で補間する
    # 実際にはビジネス的な判断が必要
    for idx in outliers.index:
        if idx > 0:
            df.at[idx, 'Close'] = df.at[idx-1, 'Close']  # 前日値で補間

# ロー、ハイ、オープンについても同様の処理が可能
# 適宜、異常値の定義や処理法はビジネスロジックに合わせる

print("異常値処理後のデータプレビュー:")
print(df.head())


欠損値数:
 Date                0
Code                0
Open                0
High                0
Low                 0
Close               0
UpperLimit          0
LowerLimit          0
Volume              0
TurnoverValue       0
AdjustmentFactor    0
AdjustmentOpen      0
AdjustmentHigh      0
AdjustmentLow       0
AdjustmentClose     0
AdjustmentVolume    0
dtype: int64
欠損値補間後:
 Date                0
Code                0
Open                0
High                0
Low                 0
Close               0
UpperLimit          0
LowerLimit          0
Volume              0
TurnoverValue       0
AdjustmentFactor    0
AdjustmentOpen      0
AdjustmentHigh      0
AdjustmentLow       0
AdjustmentClose     0
AdjustmentVolume    0
dtype: int64
異常値処理後のデータプレビュー:
         Date   Code    Open    High     Low   Close UpperLimit LowerLimit  \
0  2023-01-04  72030  1798.0  1804.0  1787.5  1799.0          0          0   
1  2023-01-05  72030  1812.0  1819.5  1793.5  1807.5          0          0   
2  

In [12]:
# 特徴量エンジニアリング

# dfは date, open, high, low, close, volume 列を持ち、日付順にソート済みとする
# date列をインデックスに設定（任意）
df = df.set_index('Date')

# 1. リターン値の算出（対数リターン or 単純リターン）
# 単純リターン
df['return'] = df['Close'].pct_change()

# 対数リターン(必要に応じて)
# df['log_return'] = np.log(df['close']).diff()

# 2. ボラティリティ指標（20日ローリング標準偏差）
df['volatility_20'] = df['return'].rolling(window=20).std()

# 3. 移動平均・移動平均乖離
df['ma_5'] = df['Close'].rolling(window=5).mean()
df['ma_20'] = df['Close'].rolling(window=20).mean()
df['ma_gap_20'] = (df['Close'] - df['ma_20']) / df['ma_20'] * 100  # 20日移動平均乖離率

# 4. RSIなどのテクニカル指標（RSI計算例：14日を想定）
# RSIは: RSI = 100 - 100/(1+RS) ここでRS = (平均上昇幅/平均下落幅)
window = 14
delta = df['Close'].diff()
up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)

roll_up = up.rolling(window=window).mean()
roll_down = down.rolling(window=window).mean()

rs = roll_up / roll_down
df['rsi_14'] = 100.0 - (100.0 / (1.0 + rs))

# 5. 出来高関連特徴量（20日移動平均）
df['volume_ma_20'] = df['Volume'].rolling(window=20).mean()
df['volume_change'] = df['Volume'].pct_change()

# 6. その他、例えばリターンの累積和でトレンド指標を作る
df['cumulative_return'] = (1 + df['return']).cumprod() - 1

# 欠損値が新たに発生する可能性がある（ローリング計算の初期部分など）
# 最終的に欠損は除去または前方補間、もしくは学習前処理で対処
df = df.dropna()

print(df.head(30))  # 確認用


             Code    Open    High     Low   Close UpperLimit LowerLimit  \
Date                                                                      
2023-02-02  72030  1908.0  1909.5  1875.5  1880.5          0          0   
2023-02-03  72030  1887.0  1895.5  1882.5  1891.5          0          0   
2023-02-06  72030  1909.0  1934.5  1908.0  1913.5          0          0   
2023-02-07  72030  1920.0  1923.5  1896.5  1897.0          0          0   
2023-02-08  72030  1902.0  1904.5  1895.5  1897.5          0          0   
2023-02-09  72030  1900.0  1923.0  1883.0  1901.0          0          0   
2023-02-10  72030  1885.5  1891.0  1869.0  1879.0          0          0   
2023-02-13  72030  1870.0  1880.0  1857.0  1876.5          0          0   
2023-02-14  72030  1894.5  1896.5  1878.5  1882.5          0          0   
2023-02-15  72030  1890.0  1893.0  1868.5  1874.0          0          0   
2023-02-16  72030  1888.5  1915.0  1884.5  1913.0          0          0   
2023-02-17  72030  1893.0